In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai\\research'

In [3]:
import os
os.chdir("./")

In [4]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai\\research'

In [5]:
import os
os.chdir("End-to-end-medical-chatbot-generative-ai")


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'End-to-end-medical-chatbot-generative-ai'

In [ ]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# #Extract Data From the PDF File
# def load_pdf_file(data):
#     loader= DirectoryLoader(data,
#                             glob="*.pdf",
#                             loader_cls=PyPDFLoader)

#     documents=loader.load()S

#     return documents

In [ ]:
# extracted_data = load_pdf_file(data="../Data")



In [ ]:
#Split the Data into Text Chunks
# def text_split(extracted_data):
#     text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
#     text_chunks=text_splitter.split_documents(extracted_data)
#     return text_chunks

In [ ]:
# text_chunks=text_split(extracted_data)
# print("Length of Text Chunks", len(text_chunks))

In [ ]:
#  text_chunksS

In [ ]:
# -------------------------------------------------------------
# IMPORTANT NOTE ABOUT `text_chunks` AND THE DATA INGESTION FLOW
# -------------------------------------------------------------
#
# When we run a RAG (Retrieval-Augmented Generation) pipeline,
# the *first* step usually looks like this:
#
#    1. Load PDF files
#    2. Split PDF content into text chunks  → `text_chunks`
#    3. Convert each chunk into embeddings
#    4. Upload embeddings into Pinecone (or any vector database)
#
# HOWEVER — in this notebook `text_chunks` is EMPTY:
#
#    print(text_chunks)  →  []
#    print(len(text_chunks))  →  0
#
# Why does this happen?
# ---------------------
# Because the local PDF file is *corrupted* and contains no
# readable text. As a result:
#
#    extracted_data = []        # no documents loaded
#    text_chunks = []           # nothing to split
#
# And therefore, we CANNOT run:
#
#    Pinecone.from_documents(text_chunks, ...)
#
# Without any text chunks, there is nothing to embed.
#
# -------------------------------------------------------------
# SO HOW DOES OUR PROJECT STILL WORK?
# -------------------------------------------------------------
# Even though the local PDF is corrupted NOW, the embeddings
# WERE ALREADY created earlier and uploaded into Pinecone.
#
# Pinecone permanently stores all vectors on the cloud.
# This means:
#
#    ✔ Our vector database already has 6,859 embeddings
#    ✔ We do NOT need to recreate chunks
#    ✔ We do NOT need to rerun embedding creation
#    ✔ We only need to *load* the existing Pinecone index
#
# Therefore, instead of:
#
#    Pinecone.from_documents(...)
#
# we correctly use:
#
#    PineconeVectorStore.from_existing_index(...)
#
# This loads previously-stored embeddings from Pinecone
# WITHOUT needing local `text_chunks` or a PDF.
#
# -------------------------------------------------------------
# WHEN SHOULD YOU REBUILD THE VECTOR DATABASE?
# -------------------------------------------------------------
# Only when:
#    - You add a new PDF
#    - You update the existing document
#    - You fix the corrupted PDF
#
# In that case, follow the FULL pipeline again:
#
#    load_pdf → split_text → create_embeddings → upload_to_pinecone
#
# -------------------------------------------------------------
# SUMMARY FOR FUTURE DEVELOPERS:
# -------------------------------------------------------------
# • If `text_chunks` is empty, you CANNOT generate new embeddings.
# • But your project will STILL WORK if embeddings ALREADY exist
#   inside Pinecone.
# • Always use `from_existing_index` when loading an existing
#   vector database.
# • Only use `from_documents` when building a brand new index.
#
# This is a crucial part of maintaining a clean and scalable
# RAG architecture.
# -------------------------------------------------------------


In [1]:
# Create Hugging Face Embeddings 


from langchain_community.embeddings import HuggingFaceEmbeddings





In [2]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [3]:
embeddings = download_hugging_face_embeddings()

C:\Users\rautl\AppData\Local\Temp\ipykernel_22156\177019967.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\rautl\miniconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [5]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [6]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [7]:
# -------------------------------------------------------------
# ❌ OLD / WRONG CODE (before):
# -------------------------------------------------------------
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec
# pc = Pinecone(api_key=PINECONE_API_KEY)
# 
# ❗ Problem:
#   - This was using Pinecone v2 / GRPC client.
#   - Your index was created in Pinecone v3 (Serverless).
#   - API key + client version mismatch → 401 Unauthorized errors.
#
# -------------------------------------------------------------
# ✔ NEW / CORRECT CODE (after):
# -------------------------------------------------------------
# Use the official Pinecone v3 client (no GRPC, no ServerlessSpec)
# This matches the version installed AND the index shown in dashboard.
# -------------------------------------------------------------

from pinecone import Pinecone       # Pinecone v3 SDK
import os
from dotenv import load_dotenv      # Load keys stored in .env file

# Load environment variables (API keys, etc.)
load_dotenv()

# Fetch Pinecone API key from .env
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Create Pinecone client using v3 SDK
# ✔ This is the correct method for Pinecone Serverless indexes
pc = Pinecone(api_key=PINECONE_API_KEY)

# Name of your existing Pinecone index (must match dashboard exactly)
index_name = "medicalbot"

# -------------------------------------------------------------
# NOTE:
# We are NOT creating a new index here because:
#   ✔ Your index already exists
#   ✔ It already contains 6859 embeddings
#   ✔ Re-creating it would overwrite or delete stored vectors
# -------------------------------------------------------------

# ❌ OLD CODE (Index Creation) — Do NOT run again:
# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )
# -------------------------------------------------------------
# END OF CONFIGURATION BLOCK
# -------------------------------------------------------------


In [8]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

os.environ["GROQ_API_KEY"] = GROQ_API_KEY



In [ ]:
# from langchain_pinecone import Pinecone

# docsearch = Pinecone.from_documents(
#     documents=text_chunks,
#     embedding=embeddings,
#     index_name=index_name
# )

In [9]:
# Load Existing index 

from langchain_pinecone import Pinecone

docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
# This step reads embeddings already stored inside Pinecone,
# without needing your PDF or text_chunks.


In [ ]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_pinecone import Pinecone as PineconeVectorStore

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



In [ ]:
# docs = docsearch.similarity_search("What is this document about?", k=3)
# docs


In [10]:
# This is optional to chequek That everything is work fine it means docsearch fetching the data from the knowledge base pinecone
query = "What is this document about?"
results = docsearch.similarity_search(query, k=3)
results



[Document(id='09a78576-c162-4376-8fc6-b75112c7fa62', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 3.0, 'page_label': '4', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='er will be corrected in future editions.\nThis book is printed on recycled paper that meets Environmental Pro-\ntection Agency standards.\nThe paper used in this publication meets the minimum requirements of\nAmerican National Standard for Information Sciences-Permanence\nPaper for Printed Library Materials, ANSI Z39.48-1984.\nThis publication is a creative work fully protected by all applicable\ncopyright laws, as well as by misappropriation, trade secret, unfair com-'),
 Document(id='3226a8d1-f4bc-4ce4-a8e7-b0408e254d05', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 3.0, 'page_label': '4', 'producer'

In [11]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [12]:
retrieved_docs = retriever.invoke("What is Acne?")

In [ ]:
retrieved_docs

[Document(id='cae53c34-60cf-47b6-9285-8455400cc6f8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='29da7ec7-f39d-476a-9ff7-882817caefc4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='42f135c5-857b-4740-a712-6228dcc20493', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 

In [13]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.4,
    max_tokens=500,
    api_key=os.getenv("GROQ_API_KEY")
)


In [14]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [15]:
# 4. Build the final LLM chain (same job as StuffDocumentsChain in old LangChain)
# This takes the prompt → runs it through the LLM → converts output to clean text
parser = StrOutputParser()
chain = prompt | llm | parser

# 5. Full RAG pipeline (same job as RetrievalChain in old LangChain)
# Step 1: Retrieve top 3 most relevant chunks from Pinecone based on user question
# Step 2: Merge those chunks into one single context block
# Step 3: Send {context + question} to the LLM and return its answer
def rag_pipeline(query):
    docs = docsearch.similarity_search(query, k=3)
    context = "\n\n".join([d.page_content for d in docs])
    return chain.invoke({"context": context, "input": query})

# 6. Test the chatbot by asking a question
rag_pipeline("What is Acne?")



'Acne is a skin disorder in which the sebaceous glands become inflamed. It is characterized by the occurrence of pimples, blackheads, and other skin lesions. Acne vulgaris is the most common form of acne.'

In [1]:
# -------------------------------------------------------------
# GROQ LLM SANITY CHECK (WHY THIS BLOCK IS IMPORTANT)
# -------------------------------------------------------------
# Before using Groq inside your RAG pipeline, we MUST test:
#   1. API key loading from .env file
#   2. Authentication with the Groq server
#   3. Whether the selected model exists and is available
#   4. Whether the client can generate a basic response
#
# If this block works → your whole chatbot will work.
# -------------------------------------------------------------

from groq import Groq
import os
from dotenv import load_dotenv

# Load environment variables (GROQ_API_KEY)
load_dotenv()

# Initialize Groq client using the API key from .env
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# -------------------------------------------------------------
# Send a simple test message to verify:
#   ✔ Correct API key
#   ✔ Correct model name (llama-3.1-8b-instant)
#   ✔ Groq servers responding properly
#   ✔ Client library works
# -------------------------------------------------------------
chat = client.chat.completions.create(
    model="llama-3.1-8b-instant",         # Groq's fast + free model
    messages=[{"role": "user", "content": "Hello"}]
)

# Print the model's response to confirm everything is working
print(chat.choices[0].message.content)

# -------------------------------------------------------------
# If you see a response (like "How can I assist you today?"):
#   ✔ Your Groq LLM is fully functional.
#   ✔ Safe to integrate with RAG pipeline.
# -------------------------------------------------------------







Hello. How can I assist you today?
